In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import fnmatch
import string
from urllib.parse import urlparse
import contractions

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/nitanshjain/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [20]:
tweets_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/tweets_v2.csv')
print(tweets_df.shape)
tweets_df.head()

(26015, 4)


,Unnamed: 0,date_created,tweet,city
0,0,2022-12-12 16:13:45+00:00,@esichq @byadavbjp @Rameswar_Teli @mygovindia ...,Mumbai
1,1,2022-12-10 06:30:56+00:00,@PotholeWarriors @CMOMaharashtra @mieknathshin...,Mumbai
2,2,2022-11-23 13:09:18+00:00,@Iam_Ayushmann Govandi is one of the Hotspot o...,Mumbai
3,3,2022-10-27 15:58:11+00:00,Till when medical negligence will exist in gov...,Mumbai
4,4,2022-07-28 03:03:15+00:00,Me being a doctor reading this\nAlso governmen...,Mumbai


In [21]:
tweets_df.rename(columns = {'Unnamed: 0':'tweet_id'}, inplace=True)
print(tweets_df.dtypes)

tweet_id         int64
date_created    object
tweet           object
city            object
dtype: object


In [22]:
#selecting only year from date_created column
tweets_df['date_created'] = pd.to_datetime(tweets_df['date_created'])
tweets_df['year'] = tweets_df['date_created'].dt.year
tweets_df.drop(['date_created'], axis=1, inplace=True)
tweets_df.head()

,tweet_id,tweet,city,year
0,0,@esichq @byadavbjp @Rameswar_Teli @mygovindia ...,Mumbai,2022
1,1,@PotholeWarriors @CMOMaharashtra @mieknathshin...,Mumbai,2022
2,2,@Iam_Ayushmann Govandi is one of the Hotspot o...,Mumbai,2022
3,3,Till when medical negligence will exist in gov...,Mumbai,2022
4,4,Me being a doctor reading this\nAlso governmen...,Mumbai,2022


In [23]:
print(tweets_df.city.value_counts())
print(tweets_df.year.value_counts())

Delhi        10005
Mumbai        6715
Hyderabad     3597
Bangalore     3229
Kolkata       1413
Chennai       1056
Name: city, dtype: int64
2020    8963
2021    7255
2019    3885
2022    3214
2018    2698
Name: year, dtype: int64


In [24]:
print('Shape of dataset before removal of duplicates is {}'.format(tweets_df.shape))
tweets_no_dupl_df = tweets_df.drop_duplicates(subset=['tweet'])
print('Shape of dataset after removal of duplicates is {}'.format(tweets_no_dupl_df.shape))

Shape of dataset before removal of duplicates is (26015, 4)
Shape of dataset after removal of duplicates is (19350, 4)


In [25]:
print(tweets_no_dupl_df.city.value_counts())
print(tweets_no_dupl_df.year.value_counts())

Delhi        7379
Mumbai       4963
Hyderabad    2614
Bangalore    2471
Kolkata      1077
Chennai       846
Name: city, dtype: int64
2020    6517
2021    5515
2019    2767
2022    2454
2018    2097
Name: year, dtype: int64


In [27]:
def removing_links(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
        
          tokens = tokenizer.tokenize(tweets)
          x = [word for word in tokens if not urlparse(word).scheme]
          tweets = ' '.join(x)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def contractions_handling(df):
     for tweets in df.loc[:,'tweet']:
          
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          tweets = contractions.fix(tweets)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def adding_space_bw_words_punc(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          tweets = tweets.replace(',', ' , ').replace('.', ' . ').replace('?', ' ? ').replace('!', ' ! ')
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df

def removing_hashtags_mentions(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          tokens = [word for word in tokens if word[0] not in ('#', '@')]
          tokens = [word for word in tokens if word[0] not in ('▪')]
          tweets = ' '.join(tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df
          
def removing_punctuations_emojis(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tweets = tweets.translate(str.maketrans('', '', string.punctuation)) # removes punctuations
          tweets = re.sub(r'[^\x00-\x7F]+', ' ', tweets) # removes emojis
          tweets = tweets.lower() # converts text to lower case
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df

def removing_numbers(df):
     for tweets in df.loc[:, 'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          #remove numbers
          tweets = re.sub(r'\d+', '', tweets)
          tweets = re.sub(' +', ' ', tweets)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
     
     return df

def removing_stopwords(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]
          tweets = ' '.join(filtered_tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def lemmatization(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          lemmatizer = WordNetLemmatizer()
          lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
          tweets = ' '.join(lemmatized_tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def removing_single_characters(df):
     for tweets in df.loc[:,'tweet']:
          
          tokenizer = TweetTokenizer()
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          tokens = tokenizer.tokenize(tweets)
          filtered_tokens = [word for word in tokens if len(word) > 1]
          tweets = ' '.join(filtered_tokens)
          
          df.loc[df['tweet_id']==tweet_id, 'tweet'] = tweets
          
     return df

def removing_tweets_less_5(df):
     for tweets in df.loc[:, 'tweet']:
          
          tweet_id = df.loc[df['tweet'] == tweets, 'tweet_id'].values[0]
          
          if len(tweets.split()) < 5:
               df.drop(df[df['tweet_id']==tweet_id].index, inplace=True)
     return df

def data_preprocessing(df):
     
     df = removing_links(df)
     df = contractions_handling(df)
     df = adding_space_bw_words_punc(df)
     df = removing_hashtags_mentions(df)
     df = removing_punctuations_emojis(df)
     df = removing_numbers(df)
     df = removing_stopwords(df)
     df = lemmatization(df)
     df = removing_single_characters(df)
     df = removing_tweets_less_5(df)
     df.reset_index(drop=True, inplace=True)
     
     return df

In [28]:
tweets_cleaned_df = data_preprocessing(tweets_no_dupl_df)
print(tweets_cleaned_df.shape)
tweets_cleaned_df.head()

/var/folders/mc/_plzx65559n4fxtnmj3tqk9w0000gp/T/ipykernel_4103/3301604366.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['tweet_id']==tweet_id].index, inplace=True)


(18996, 4)


,tweet_id,tweet,city,year
0,0,bad thing say government medical spare operati...,Mumbai,2022
1,1,netas family admitted government hospital priv...,Mumbai,2022
2,2,govandi one hotspot well respiratory disease c...,Mumbai,2022
3,3,till medical negligence exist government hospi...,Mumbai,2022
4,4,doctor reading also government hospital resident,Mumbai,2022


In [29]:
# creating csv of cleaned dataset
pd.DataFrame.to_csv(tweets_cleaned_df, '/Users/nitanshjain/Documents/Projects/Twitter_Data_Analysis/v2/data/tweets_cleaned_v2.csv', index=False)